In [1]:
# Donwload libraries
!pip install scikit-learn
!pip install seaborn
!pip install numpy
!pip install pandas
!pip install scikit-multilearn
!pip install kagglehub
!git lfs install

Updated Git hooks.
Git LFS initialized.


Importing libraries and packages

In [2]:
#  git lfs track "*.csv" to track all csv files

import os
import kagglehub
import sqlite3
#Data Wrangling

import numpy as np
import pandas as pd

#Trees
from sklearn import tree
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

#Preprocessing Packages
from sklearn.model_selection import train_test_split

#Cross Validation
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

#Model tuning
from sklearn.model_selection import GridSearchCV

#Metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix

#Plotting
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

Loading Data Sets

In [3]:
# set up respective path to the data
project_root = os.path.abspath(os.path.join(".."))
data_path = os.path.join(project_root, "raw-data/")

# concatenate all seasons of data in a single df
df=pd.DataFrame()
for file in os.listdir(data_path):
    file_path=os.path.join(data_path,file)
    data=pd.read_csv(file_path)
    df=pd.concat([df,data],ignore_index=True)
    print(file)


NBA_2009_Shots.csv
NBA_2004_Shots.csv
NBA_2010_Shots.csv
NBA_2016_Shots.csv
NBA_2023_Shots.csv
NBA_2008_Shots.csv
NBA_2024_Shots.csv
NBA_2011_Shots.csv
NBA_2005_Shots.csv
NBA_2017_Shots.csv
NBA_2022_Shots.csv
NBA_2006_Shots.csv
NBA_2012_Shots.csv
NBA_2019_Shots.csv
NBA_2021_Shots.csv
NBA_2014_Shots.csv
NBA_2013_Shots.csv
NBA_2007_Shots.csv
NBA_2018_Shots.csv
NBA_2020_Shots.csv
NBA_2015_Shots.csv


In [4]:
# inspect the Leagues data set
print("Number of duplicate rows: " + str(df.duplicated().sum()))
print("Leagues DF dtypes:")
print(df.dtypes)
print("----------------------------\nLeagues DF describe:")
print(df.describe())
print("----------------------------\nCheck for Null values:")
print(df.isna().sum())
print("----------------------------\nFirst 5 row of the dataset")
df.head()

Number of duplicate rows: 192
Leagues DF dtypes:
SEASON_1            int64
SEASON_2           object
TEAM_ID             int64
TEAM_NAME          object
PLAYER_ID           int64
PLAYER_NAME        object
POSITION_GROUP     object
POSITION           object
GAME_DATE          object
GAME_ID             int64
HOME_TEAM          object
AWAY_TEAM          object
EVENT_TYPE         object
SHOT_MADE            bool
ACTION_TYPE        object
SHOT_TYPE          object
BASIC_ZONE         object
ZONE_NAME          object
ZONE_ABB           object
ZONE_RANGE         object
LOC_X             float64
LOC_Y             float64
SHOT_DISTANCE       int64
QUARTER             int64
MINS_LEFT           int64
SECS_LEFT           int64
dtype: object
----------------------------
Leagues DF describe:
           SEASON_1       TEAM_ID     PLAYER_ID       GAME_ID         LOC_X  \
count  4.231262e+06  4.231262e+06  4.231262e+06  4.231262e+06  4.231262e+06   
mean   2.014191e+03  1.610613e+09  4.136611e+05  2.13

,SEASON_1,SEASON_2,TEAM_ID,TEAM_NAME,PLAYER_ID,PLAYER_NAME,POSITION_GROUP,POSITION,GAME_DATE,GAME_ID,...,BASIC_ZONE,ZONE_NAME,ZONE_ABB,ZONE_RANGE,LOC_X,LOC_Y,SHOT_DISTANCE,QUARTER,MINS_LEFT,SECS_LEFT
0,2009,2008-09,1610612744,Golden State Warriors,201627,Anthony Morrow,G,SG,04-15-2009,20801229,...,Restricted Area,Center,C,Less Than 8 ft.,-0.0,5.25,0,4,0,1
1,2009,2008-09,1610612744,Golden State Warriors,101235,Kelenna Azubuike,F,SF,04-15-2009,20801229,...,Restricted Area,Center,C,Less Than 8 ft.,-0.0,5.25,0,4,0,9
2,2009,2008-09,1610612756,Phoenix Suns,255,Grant Hill,F,SF,04-15-2009,20801229,...,Restricted Area,Center,C,Less Than 8 ft.,-0.0,5.25,0,4,0,25
3,2009,2008-09,1610612739,Cleveland Cavaliers,200789,Daniel Gibson,G,PG,04-15-2009,20801219,...,Restricted Area,Center,C,Less Than 8 ft.,-0.2,5.25,0,5,0,4
4,2009,2008-09,1610612756,Phoenix Suns,255,Grant Hill,F,SF,04-15-2009,20801229,...,Mid-Range,Left Side,L,8-16 ft.,8.7,7.55,8,4,1,3


In [5]:
df_clean = df.drop_duplicates()
df_clean.head()

,SEASON_1,SEASON_2,TEAM_ID,TEAM_NAME,PLAYER_ID,PLAYER_NAME,POSITION_GROUP,POSITION,GAME_DATE,GAME_ID,...,BASIC_ZONE,ZONE_NAME,ZONE_ABB,ZONE_RANGE,LOC_X,LOC_Y,SHOT_DISTANCE,QUARTER,MINS_LEFT,SECS_LEFT
0,2009,2008-09,1610612744,Golden State Warriors,201627,Anthony Morrow,G,SG,04-15-2009,20801229,...,Restricted Area,Center,C,Less Than 8 ft.,-0.0,5.25,0,4,0,1
1,2009,2008-09,1610612744,Golden State Warriors,101235,Kelenna Azubuike,F,SF,04-15-2009,20801229,...,Restricted Area,Center,C,Less Than 8 ft.,-0.0,5.25,0,4,0,9
2,2009,2008-09,1610612756,Phoenix Suns,255,Grant Hill,F,SF,04-15-2009,20801229,...,Restricted Area,Center,C,Less Than 8 ft.,-0.0,5.25,0,4,0,25
3,2009,2008-09,1610612739,Cleveland Cavaliers,200789,Daniel Gibson,G,PG,04-15-2009,20801219,...,Restricted Area,Center,C,Less Than 8 ft.,-0.2,5.25,0,5,0,4
4,2009,2008-09,1610612756,Phoenix Suns,255,Grant Hill,F,SF,04-15-2009,20801229,...,Mid-Range,Left Side,L,8-16 ft.,8.7,7.55,8,4,1,3


In [6]:
#TODO:
print("Number of rows containing NA values:\n" + str(df_clean.isna().sum()[df_clean.isna().sum()>0]))
df_clean[df_clean["POSITION_GROUP"].isna() | df_clean["POSITION"].isna()]

Number of rows containing NA values:
POSITION_GROUP    7930
POSITION          7930
dtype: int64


,SEASON_1,SEASON_2,TEAM_ID,TEAM_NAME,PLAYER_ID,PLAYER_NAME,POSITION_GROUP,POSITION,GAME_DATE,GAME_ID,...,BASIC_ZONE,ZONE_NAME,ZONE_ABB,ZONE_RANGE,LOC_X,LOC_Y,SHOT_DISTANCE,QUARTER,MINS_LEFT,SECS_LEFT
812184,2023,2022-23,1610612761,Toronto Raptors,1628384,O.G. Anunoby,NaN,NaN,10-19-2022,22200008,...,Above the Break 3,Left Side Center,LC,24+ ft.,15.0,26.95,26,1,11,23
812187,2023,2022-23,1610612761,Toronto Raptors,1628384,O.G. Anunoby,NaN,NaN,10-19-2022,22200008,...,In The Paint (Non-RA),Left Side,L,8-16 ft.,5.6,14.45,10,1,9,42
812199,2023,2022-23,1610612761,Toronto Raptors,1628384,O.G. Anunoby,NaN,NaN,10-19-2022,22200008,...,Above the Break 3,Right Side Center,RC,24+ ft.,-8.9,29.35,25,1,1,15
812207,2023,2022-23,1610612761,Toronto Raptors,1628384,O.G. Anunoby,NaN,NaN,10-19-2022,22200008,...,In The Paint (Non-RA),Center,C,Less Than 8 ft.,2.1,10.05,5,2,9,24
812222,2023,2022-23,1610612761,Toronto Raptors,1628384,O.G. Anunoby,NaN,NaN,10-19-2022,22200008,...,Restricted Area,Center,C,Less Than 8 ft.,-0.1,4.65,0,2,1,54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4157912,2015,2014-15,1610612766,Charlotte Hornets,203106,Jeffery Taylor,NaN,NaN,12-26-2014,21400438,...,Restricted Area,Center,C,Less Than 8 ft.,-0.1,7.45,2,4,2,49
4157940,2015,2014-15,1610612766,Charlotte Hornets,203106,Jeffery Taylor,NaN,NaN,12-26-2014,21400438,...,Mid-Range,Right Side,R,8-16 ft.,-11.9,5.05,11,4,4,5
4158003,2015,2014-15,1610612766,Charlotte Hornets,203106,Jeffery Taylor,NaN,NaN,12-26-2014,21400438,...,Restricted Area,Center,C,Less Than 8 ft.,-1.5,5.05,1,4,6,59
4162561,2015,2014-15,1610612766,Charlotte Hornets,203106,Jeffery Taylor,NaN,NaN,12-22-2014,21400409,...,Right Corner 3,Right Side,R,24+ ft.,-23.6,6.35,23,4,2,14


In [7]:
conn = sqlite3.connect("database.db")

df_clean.to_sql("nba", conn, if_exists="replace", index=False)


4231070

In [8]:
conn.close()